In [35]:
import pandas as pd
from utils import data_util as du, preproc_utils as pu, extractor_util as eu
import importlib
import os

In [4]:
DATA_SRC_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data'
EXT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external'
INT_DATA_DIR = '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/internal'

In [11]:
# Usage example
dataset = du.DatasetLoader(EXT_DATA_DIR)
brandenburg = dataset.load_scene('brandenburg_gate')


In [14]:
# Access data
images_path = brandenburg.images_dir
calibration_data = brandenburg.calibration

In [ ]:
for i in images_path.iterdir():
    print(i)

In [16]:
# Usage
config = pu.PreprocessConfig(size=(512, 512), normalize=False)
preprocessor = pu.ImagePreprocessor(config)

In [66]:
# Get two images from images_path
images = list(images_path.iterdir())[:2]
image1, image2 = images[0], images[1]
print(image1, image2)

/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/train/brandenburg_gate/images/10624116_2746543073.jpg /Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/data/external/train/brandenburg_gate/images/96174448_2036661198.jpg


In [68]:
img1 = preprocessor.process_image(image1)
img2 = preprocessor.process_image(image2)

In [51]:
default_sift_config = eu.SIFTConfig()
extractor_config = eu.FeatureExtractorConfig(default_sift_config)
extractor = eu.FeatureExtractor(config=extractor_config)

In [69]:
i1f = extractor.extract_features(img1)
i2f = extractor.extract_features(img2)

In [61]:
importlib.reload(eu)

<module 'utils.extractor_util' from '/Users/yoav/Documents/Yoav/CS/22928 - Intro to CV/cv-final/utils/extractor_util.py'>

In [70]:
bf_config = eu.BFMatcherConfig()
matcher_config = eu.MatcherConfig(bf_config)

In [63]:
matcher = eu.FeatureMatcher(matcher_config)

In [71]:
matches = matcher.match_features(i1f, i2f)

In [72]:
matches

[< cv2.DMatch 0x10ffead70>,
 < cv2.DMatch 0x10ffeb4d0>,
 < cv2.DMatch 0x10ffe9fb0>,
 < cv2.DMatch 0x10ffe9db0>,
 < cv2.DMatch 0x10ffe9c30>,
 < cv2.DMatch 0x10ffeac90>,
 < cv2.DMatch 0x10ffebbd0>,
 < cv2.DMatch 0x10ffebef0>,
 < cv2.DMatch 0x10ffe9e90>,
 < cv2.DMatch 0x10e337070>,
 < cv2.DMatch 0x10ffea4f0>,
 < cv2.DMatch 0x10ffea7b0>,
 < cv2.DMatch 0x110094210>,
 < cv2.DMatch 0x10ffeab70>,
 < cv2.DMatch 0x10ffeacf0>,
 < cv2.DMatch 0x10ffeadb0>,
 < cv2.DMatch 0x10ffe9df0>,
 < cv2.DMatch 0x110094530>,
 < cv2.DMatch 0x10ffead90>,
 < cv2.DMatch 0x10ffea710>,
 < cv2.DMatch 0x10ffea2b0>,
 < cv2.DMatch 0x10e1d1b90>,
 < cv2.DMatch 0x10ffea590>,
 < cv2.DMatch 0x10ffe8d10>,
 < cv2.DMatch 0x10ffeb7f0>,
 < cv2.DMatch 0x10ffea3d0>,
 < cv2.DMatch 0x1100940f0>,
 < cv2.DMatch 0x10ffeaf70>,
 < cv2.DMatch 0x10ffeabf0>,
 < cv2.DMatch 0x10ffea3f0>,
 < cv2.DMatch 0x10ffea290>,
 < cv2.DMatch 0x10ffeb030>,
 < cv2.DMatch 0x10e7c4370>,
 < cv2.DMatch 0x110094390>,
 < cv2.DMatch 0x10ffea8d0>,
 < cv2.DMatch 0x10ff